In this notebook I will plot all the longidutinal data for each individual separately

In [1]:
import src.data.breathe_data as br
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import src.data.helpers as dh
from datetime import timedelta
import src.o2_fev1_analysis.smooth as smooth
import numpy as np

In [2]:
# df = br.build_O2_FEV1_FEF2575_df(meas_file=2, remove_nan=False)
df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan")
df["PEF (L/s)"] = df.PEF / 60

In [3]:
# df.to_excel(f"{dh.get_path_to_main()}ExcelFiles/BR/BR_O2_FEV1_FEF2575_PEF_Nan.xlsx", index=False)

In [27]:
def apply_new_smoothing(df, col, scale, shift):
    df[f"ec{col}_2"] = df[col]
    no_nan_mask = ~df[f"ec{col}_2"].isna()
    df.iloc[no_nan_mask] = smooth.identify_and_replace_outliers_up(
        df[no_nan_mask], f"ec{col}_2", scale=scale, shift=shift
    ).copy()
    df[f"ec{col}_3"] = df[f"ec{col}_2"]
    df[f"ec{col}_3"][no_nan_mask] = smooth.smooth_vector(
        df[f"ec{col}_2"][no_nan_mask].to_numpy(), "max"
    )

    return df


df1 = (
    df.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "FEV1", 3, 0.5))
    .reset_index(drop=True)
)
df1 = (
    df1.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "FEF2575", 3, 0.5))
    .reset_index(drop=True)
)
df1 = (
    df1.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "PEF (L/s)", 3, 1))
    .reset_index(drop=True)
)


def plot_profile_for_ID(df_for_ID):
    df_for_ID = df_for_ID.reset_index()
    fig = make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.04,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["O2 Saturation"], x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(y=df_for_ID.FEV1, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=2, col=1)
    fig.add_trace(
        go.Scatter(y=df_for_ID.ecFEV1_3, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.FEF2575,
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=3,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=3, col=1)
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.ecFEF2575_3, x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["PEF (L/s)"],
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=4,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=4, col=1)
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["ecPEF (L/s)_3"], x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=4,
        col=1,
    )
    fig.update_yaxes(
        title="SpO2<br>(%)",
        row=1,
        col=1,
        range=[
            np.nanmin(df["O2 Saturation"]) * 0.98,
            np.nanmax(df["O2 Saturation"]) * 1.02,
        ],
    )
    fig.update_yaxes(
        title="FEV1<br>(L)",
        row=2,
        col=1,
        range=[np.nanmin(df.FEV1) * 0.98, np.nanmax(df.FEV1) * 1.02],
    )
    fig.update_yaxes(
        title="FEF2575<br>(L/s)",
        row=3,
        col=1,
        range=[np.nanmin(df.FEF2575) * 0.98, np.nanmax(df.FEF2575) * 1.02],
    )
    fig.update_yaxes(
        title="PEF<br>(L/s)",
        row=4,
        col=1,
        range=[np.nanmin(df.PEF) / 60 * 0.98, np.nanmax(df.PEF) / 60 * 1.02],
    )
    fig.update_yaxes(nticks=10)
    fig.update_traces(marker=dict(size=3))
    title = f"Measures for ID {df_for_ID.ID[0]}"
    fig.update_layout(
        font=dict(size=8), showlegend=False, title=title, height=1000, width=1400
    )
    # fig.show()
    fig.write_image(f"{dh.get_path_to_main()}PlotsBreathe/ID_profiles/{title}.pdf")
    return


df1.groupby(by="ID").apply(plot_profile_for_ID)

ID 101 - Outlier up for ecFEV1_2, day 2023-10-24: 2.52 > 2.19 and > 2.22 (mean=1.72,std=0.16), update to 1.64
ID 105 - Outlier up for ecFEV1_2, day 2019-01-18: 2.19 > 1.76 and > 2.08 (mean=1.58,std=0.06), update to 2.01
ID 107 - Outlier up for ecFEV1_2, day 2023-09-19: 2.71 > 2.32 and > 2.29 (mean=1.78,std=0.18), update to 1.66
ID 123 - Outlier up for ecFEV1_2, day 2019-09-18: 3.86 > 3.76 and > 3.72 (mean=3.22,std=0.18), update to 3.21
ID 146 - Outlier up for ecFEV1_2, day 2019-10-26: 1.56 > 1.23 and > 1.53 (mean=1.03,std=0.07), update to 2.24
ID 221 - Outlier up for ecFEV1_2, day 2022-08-09: 3.03 > 2.45 and > 2.44 (mean=1.94,std=0.17), update to 2.06
ID 244 - Outlier up for ecFEV1_2, day 2022-08-27: 4.38 > 3.86 and > 3.61 (mean=3.11,std=0.25), update to 3.06
ID 104 - Outlier up for ecFEF2575_2, day 2023-07-26: 1.73 > 1.72 and > 1.67 (mean=1.17,std=0.18), update to 1.12
ID 105 - Outlier up for ecFEF2575_2, day 2019-01-18: 2.48 > 1.81 and > 1.99 (mean=1.49,std=0.11), update to 2.47
ID 1

""
